# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(index=df["prestige"], columns = df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [4]:
dummy_ranks = pd.get_dummies(df['prestige'])

dummy_ranks.rename(columns={1.0:'prestige_1',2.0:'prestige_2',3.0:'prestige_3',4.0:'prestige_4'}, inplace=True)
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: we need n-1 variables so for this data set this would be 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [5]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
#print handCalc.columns
#handCalc.rename(columns={1.0:'prestige_1'}, inplace=True)
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [6]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])


admit,0,1
prestige_1,,
0.0,243,93
1.0,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [61]:
#percentage of rank 1 students admitted
print np.true_divide(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][1],(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[0][1]+pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][1]))
print(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][1])
#odds of being admitted
#print np.true_divide(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[0][1],pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][1])

0.540983606557
33


The total percentage admitted to grad school if you attended a rank one school is the number of admitted students devided by the total number of students, in this case that is 33/61 = 0.54

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [68]:
#percentage of non rank 1 students admitted
print np.true_divide(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][0],(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[0][0]+pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][0]))

#odds of being admitted if not in rank 1 school
#print np.true_divide(pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[1][0],pd.crosstab(index=handCalc["prestige_1"], columns = df['admit'])[0][0])

0.276785714286


#### 3.3 Calculate the odds ratio

In [69]:
0.540983606557/0.276785714286 # pretty sure I did something wrong here....

1.9545214172361758

#### 3.4 Write this finding in a sentenance: 

Answer: when going to a prestige 1 school you are ~twice as likely to get accepted into university then from other insitutions

#### 3.5 Print the cross tab for prestige_4

In [10]:
pd.crosstab(index=handCalc["prestige_4"], columns = df['admit'])

admit,0,1
prestige_4,,
0.0,216,114
1.0,55,12


#### 3.6 Calculate the OR 

In [11]:
(12./67.)/(114./330.)

0.5184603299293008

#### 3.7 Write this finding in a sentence

Answer:When goint to a prestige 4 school you are only half as likly to get into university then poeple from more prestigues schools

## Part 4. Analysis

In [87]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [88]:
# manually add the intercept
data['intercept'] = 1.0
data.head()
data = sm.add_constant(data)

#### 4.1 Set the covariates to a variable called train_cols

In [89]:
data.cov()
train_cols = data.cov()
print data.columns

Index([u'admit', u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')


#### 4.2 Fit the model

In [91]:
#sklearn or sm??

model = sm.Logit(data['admit'], data[['gre', 'gpa' ,'prestige_2', 'prestige_3', 'prestige_4','intercept']]).fit()

#model = LogisticRegression()

#y = data['admit'].values
#X = data[['gre','gpa', 'prestige_2', 'prestige_3', 'prestige_4']].values


#model.fit(X,y)

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [92]:
#help(sm.regression.linear_model)
#help(sm.OLS)
print(model.summary())

#print((model.coef_))

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 27 Apr 2018   Pseudo R-squ.:                 0.08166
Time:                        15:56:47   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2    -0.6801      0.317     -2.146      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [93]:
# this is how it would have worked with sm but not with sklearn

from __future__ import division
conf_95=model.conf_int()[1]

coef=(model.params.values) # becasue it is the log odds ??

ratio = coef/(conf_95)

In [94]:
ratio

gre            0.508533
gpa            0.544589
prestige_2    11.540179
prestige_3     2.019804
prestige_4     2.112918
intercept      2.367355
Name: 1, dtype: float64

#### 4.5 Interpret the OR of Prestige_2

going to a pres 2 school would have an 11 times higher chance then going to a pres 3 or pres 4 school

#### 4.6 Interpret the OR of GPA

For any point increase in GPA there is a 0.5%increase in the chance of being admitted


## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [95]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [96]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.rename(columns={0:'gre',1:'gpa',2:'prestige',3:'intercept'}, inplace=True)
print combos

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]
       gre       gpa  prestige  intercept
0    220.0  2.260000       1.0        1.0
1    220.0  2.260000       2.0        1.0
2    220.0  2.260000       3.0        1.0
3    220.0  2.260000       4.0        1.0
4    220.0  2.453333       1.0        1.0
5    220.0  2.453333       2.0        1.0
6    220.0  2.453333       3.0        1.0
7    220.0  2.453333       4.0        1.0
8    220.0  2.646667       1.0        1.0
9    220.0  2.646667       2.0        1.0
10   220.0  2.646667       3.0        1.0
11   220.0  2.646667       4.0        1.0
12   220.0  2.840000       1.0        1.0
13   220.0  2.840000       2.0        1.0
14   220.0  2.840000       3.0        1.0
15   220.0  2.840000       4.0        1.0
16   220.0  3.033333

/anaconda/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


#### 5.1 Recreate the dummy variables

In [97]:
# recreate the dummy variables
dummy_var = pd.get_dummies(combos['prestige'])
dummy_var.rename(columns={1.0:'prestige_1',2.0:'prestige_2',3.0:'prestige_3',4.0:'prestige_4'}, inplace=True)
dummy_var.head()
# keep only what we need for making predictions

cols_to_keep = [ 'gre', 'gpa']
combo_new = combos[cols_to_keep].join(dummy_var.ix[:, 'prestige_2':])
combo_new=combo_new.values

#### 5.2 Make predictions on the enumerated dataset

In [98]:

new_model_output = model.predict(combo_new)
pd.unique(new_model_output)

new_model_output[-4:]

ValueError: shapes (400,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)

#### 5.3 Interpret findings for the last 4 observations

Answer: Only one of the last four is going to get admitted

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.